# Лабораторная работа 3. 
# Сетевые алгоритмы. Динамические алгоритмы поиска путей.


## Выполнил студент группы Возжин Сергей Дмитриевич БПИ2301
***

### Задание

1.  Реализовать алгоритм поиска кратчайшего расстояния между двумя вершинами ориентированного взвешенного графа в соответствии с вариантом. 

2.  Предусмотреть задание графа в виде матрицы смежности/инцидентности, читаемой из файла, либо графически с помощью пользовательского интерфейса. 

3.  Разработать графический интерфейс пользователя с визуализацией графа и отображением кратчайшего расстояния между задаваемыми пользователем вершинами.

4. По результатам работы проанализировать временную сложность работы заданного алгоритма в зависимости от числа узлов и ребер графа.
Данные представить в виде таблицы.



 ### Алгоритмы:

Алгоритм Флойда-Уоршелла| Алгоритм Дейкстры | Алгоритм Беллмана-Форда | Алгоритм Джонсона| Алгоритм Левита | Алгоритм Йена



### Выполнение:

In [2]:
import math

from load_incidence_matrix import load_incidence_matrix_from_file
from load_matrix import load_graph_from_file

def bellman_ford(graph, source):
    n = len(graph)
    distance = [math.inf] * n
    distance[source] = 0

    for _ in range(n - 1):
        for u in range(n):
            for v in range(n):
                if graph[u][v] != math.inf:
                    if distance[u] + graph[u][v] < distance[v]:
                        distance[v] = distance[u] + graph[u][v]

    for u in range(n):
        for v in range(n):
            if graph[u][v] != math.inf:
                if distance[u] + graph[u][v] < distance[v]:
                    raise ValueError("Граф содержит отрицательный цикл!")

    return distance


def dijkstra_with_path(graph, start, end):
    n = len(graph)
    distances = [math.inf] * n
    previous = [None] * n
    visited = [False] * n
    distances[start] = 0

    for _ in range(n):
        min_distance = math.inf
        u = -1
        for i in range(n):
            if not visited[i] and distances[i] < min_distance:
                min_distance = distances[i]
                u = i
        if u == -1:
            break
        visited[u] = True
        for v in range(n):
            if graph[u][v] != math.inf and not visited[v]:
                new_distance = distances[u] + graph[u][v]
                if new_distance < distances[v]:
                    distances[v] = new_distance
                    previous[v] = u

    # Восстановление пути
    path = []
    if distances[end] != math.inf:
        current = end
        while current is not None:
            path.insert(0, current)
            current = previous[current]
        return distances[end], path
    else:
        return None, []

def johnson(graph, start, end):
    n = len(graph)
    new_graph = [row + [math.inf] for row in graph]
    new_graph.append([0] * n + [0])
    h = bellman_ford(new_graph, n)

    reweighted_graph = [[math.inf] * n for _ in range(n)]
    for u in range(n):
        for v in range(n):
            if graph[u][v] != math.inf:
                reweighted_graph[u][v] = graph[u][v] + h[u] - h[v]

    distance, path = dijkstra_with_path(reweighted_graph, start, end)
    if distance is None:
        return None, []
    true_distance = distance + h[end] - h[start]
    return true_distance, path


### Вывод

	1.	Беллман-Форд: O(n * m)
	2.	Перевзвешивание: O(m)
	3.	Один запуск Дейкстры: O(m log n)

Суммарная сложность:

O(n * m) + O(m) + O(m * log n) = O(n * m + m * log n)

| Размер графа  | Параметры \((n, m)\) | Оценка сложности               | Примерное время работы |
|---------------|-----------------------|--------------------------------|------------------------|
| Малый граф    | \(n = 10, m = 20\)   | \(O(n \cdot m + m \log n) = O(10 \cdot 20 + 20 \log 10)\) ≈ \(O(300)\) операций | Миллисекунды           |
| Средний граф  | \(n = 1000, m = 5000\) | \(O(1000 \cdot 5000 + 5000 \log 1000)\) ≈ \(O(5.1 \times 10^6)\) операций | Секунды                |
| Большой граф  | \(n = 10^5, m = 10^6\) | \(O(10^5 \cdot 10^6 + 10^6 \log 10^5)\) ≈ \(O(1.1 \times 10^9)\) операций | Минуты                 |